In [8]:
cd ../..

e:\


In [9]:
import torch
import time
import os
import copy

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torchvision import models, transforms
from modules.training_util import PytorchDataset, PytorchTraining

In [10]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/efficientnet/models//'
MODEL_LAST_EPOCH = '.pth'

# CONFIG VARIABLES
DATA_DIR = 'E:/Datasets/cor-splits/sgkf-8-1-1-even'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 31
LR = 0.001
LR_EPOCH_SCHEDULE = 10
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup

In [11]:
working_folder = 'notebooks/vit'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
# curr_time = datetime(curr_time.year, curr_time.month, curr_time.day, curr_time.hour)
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [12]:
# vit_b_16
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [13]:
def train_model(model, criterion, optimizer, scheduler, output_folder, start_epoch=1, num_epochs=25, epoch_save_interval=2):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    log_path = f"{output_folder}/log.txt"
    
    run_info = 'Dataset {}\nLearning Rate Epoch Schedule = {}\nLearning Rate Gamma = {}'.format(DATA_DIR, LR_EPOCH_SCHEDULE, LR_GAMMA)
            
    if not os.path.exists(log_path): 
        log = open(log_path,'x')
        log.writelines('=' * 10+'\n')
        log.writelines('\n'+run_info+'\n')
        log.writelines('=' * 10+'\n')
        log.close()        
        
    for epoch in range(start_epoch, num_epochs+1):
        epoch_start = time.time()
        
        epoch_info = 'Epoch {}/{}'.format(epoch, num_epochs)
        
        with open(log_path,'a') as log:
            log.writelines('\n'+epoch_info+'\n')
            log.writelines('-' * 10+'\n')
        
        print(epoch_info)
        print('-' * 10)
        

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            loss_acc_info = '{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)
            with open(log_path,'a') as log:
                log.writelines(loss_acc_info+'\n')
            print(loss_acc_info)
            
            if epoch % epoch_save_interval == 0:
                torch.save(model.state_dict(), f'{output_folder}/epoch_{epoch}.pth')
                
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        epoch_end = time.time() - epoch_start
        epoch_duration_info = 'Epoch duration: {:.0f} m {:.0f}s'.format(epoch_end//60, epoch_end%60)
        with open(log_path,'a') as log:
            log.writelines(epoch_duration_info+'\n')

        print(epoch_duration_info)
        print()
    
    torch.save(model.state_dict(), f'{output_folder}/last.pth')
    
    time_elapsed = time.time() - since
    
    complete_text = 'Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)
    print(complete_text)
    best_acc_text = 'Best val Acc: {:4f}'.format(best_acc)
    print(best_acc_text)
    
    with open(log_path,'a') as log:
        log.writelines(complete_text+'\n')
        log.writelines(best_acc_text+'\n')
        
    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), f'{output_folder}/best.pth')
    
    return model

In [14]:
model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = True

# A newly defined layer is created with requires_grad=True by default
model.heads = nn.Sequential(
    nn.Linear(in_features=768, out_features=2)
    )

if CONTINUE_TRAINING:
    output_folder = MODEL_WEIGHTS_PATH
    model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
else:
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
       
model = model.to(device)

criterion = nn.CrossEntropyLoss()

#optimizer = optim.SGD(model.heads.parameters(), lr=LR)
optimizer = optim.Adam(model.parameters(), lr=LR)

# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)

dataset = PytorchDataset(DATA_DIR, data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)

model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/31
----------
